In [29]:
import os
os.chdir('drive/MyDrive/ColabNotebooks/CANN_Pietro/Test')

FileNotFoundError: ignored

In [30]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [180]:

class GeneralizedInvariantsComputer(torch.nn.Module):

  def __init__(self, Vectorize=True, *args, **kwargs) -> None:
      super(GeneralizedInvariantsComputer,self).__init__(*args, **kwargs)

      self.Vectorize = Vectorize

  def forward(self,Lgen_,C_):


    if self.Vectorize == True:

      IJgen_ = self.IJgen_Computation_Vec(Lgen_,C_)

    else:

      IJgen_ = self.IJgen_Computation(Lgen_,C_)


    return IJgen_

  def IJgen_Computation(self,Lgen_,C_):

    # C_ --> deformation tensor [?,3,3]
    # Lgen_ --> generalized structural tensor vector [?,R,3,3]

    R = Lgen_.shape[1]
    batchsize = Lgen_.shape[0]


    IJgen_ = torch.zeros((batchsize,R,2))

    for data_index,(Lgen_set,C) in enumerate(zip(Lgen_,C_)):

      Cit = torch.transpose(torch.inverse(C),0,1)

      detCCit = torch.det(torch.matmul(C,Cit))

      for Lgen_index,Lgen in enumerate(Lgen_set):

        IJgen_[data_index,Lgen_index,0] = torch.trace(torch.matmul(C,Lgen))

        IJgen_[data_index,Lgen_index,1] = torch.trace(detCCit*Lgen)

    return IJgen_

  def IJgen_Computation_Vec(self,Lgen_,C_):

    # C_ --> deformation tensor [?,3,3]
    # Lgen_ --> generalized structural tensor vector [?,R,3,3]

    Igen = lambda C,Lgen : torch.trace(torch.matmul(C,Lgen))
    Jgen = lambda dC,Lgen : torch.trace(dC*Lgen)

    def Compute_dC (C):
      
      Cit = torch.transpose(torch.inverse(C),0,1)

      dC = torch.det(torch.matmul(C,Cit))

      return dC

    def Compute (Lgenset):

      comp_dC = torch.vmap(Compute_dC)
      dC_ = comp_dC(C_)

      comp_Igen = torch.vmap(Igen)
      comp_Jgen = torch.vmap(Jgen)

      Igen_ = comp_Igen(C_,Lgenset)
      Jgen_ = comp_Jgen(dC_,Lgenset)

      return Igen_,Jgen_

    vectorized = torch.vmap(Compute,in_dims=1)

    I_,J_ = vectorized(Lgen_)

    I_ = torch.transpose(I_,1,0)
    J_ = torch.transpose(J_,1,0)

    I_ = torch.unsqueeze(I_,2)
    J_ = torch.unsqueeze(J_,2)

    IJ_ = torch.concatenate((I_,J_),axis=2)

    return IJ_



In [181]:
L = torch.rand((3,4,3,3))
C = torch.rand((3,3,3))

In [182]:
model = GeneralizedInvariantsComputer()
model(L,C)

tensor([[[1.8473, 1.0262],
         [1.7485, 1.7148],
         [1.8407, 1.3588],
         [2.2728, 1.8567]],

        [[1.2115, 1.7558],
         [1.5901, 0.3308],
         [1.4613, 0.7071],
         [1.4755, 0.7973]],

        [[1.7236, 1.2140],
         [2.0223, 1.3852],
         [1.6099, 1.7654],
         [1.6063, 1.4779]]])

In [183]:
model2 = GeneralizedInvariantsComputer(Vectorize=False)
model2(L,C)

tensor([[[1.8473, 1.0262],
         [1.7485, 1.7148],
         [1.8407, 1.3588],
         [2.2728, 1.8567]],

        [[1.2115, 1.7558],
         [1.5901, 0.3308],
         [1.4613, 0.7071],
         [1.4755, 0.7973]],

        [[1.7236, 1.2140],
         [2.0223, 1.3852],
         [1.6099, 1.7654],
         [1.6063, 1.4779]]])

In [185]:
model2(L,C)[:,:,0]

tensor([[1.8473, 1.7485, 1.8407, 2.2728],
        [1.2115, 1.5901, 1.4613, 1.4755],
        [1.7236, 2.0223, 1.6099, 1.6063]])

In [186]:
model(L,C)[:,:,0]

tensor([[1.8473, 1.7485, 1.8407, 2.2728],
        [1.2115, 1.5901, 1.4613, 1.4755],
        [1.7236, 2.0223, 1.6099, 1.6063]])